#### Validation 1. Is Scaling meaningful?

In [136]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [137]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.iloc[:,1:]
test = test.iloc[:,1:]

In [138]:
train

,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,T1c,...,Height,Weight,FEV1_FVC_P,DLCO_VA_P,EGFR mutation Detection,Operation,Chemotherapy,Radiation Therapy,Survival period,Death
0,72,1,1,0,1,0,0,0,0,0,...,142.0,68.2,87,102,99,1,0,0,157,0
1,66,1,0,0,0,0,1,0,0,0,...,179.4,61.1,54,112,99,0,1,0,567,0
2,75,1,1,0,1,0,0,0,0,0,...,169.8,73.8,79,73,99,0,1,0,253,0
3,77,1,0,1,0,0,1,0,0,0,...,158.5,79.1,78,137,1,0,0,0,266,0
4,85,1,0,0,0,0,0,0,0,1,...,172.3,44.3,48,85,99,1,0,1,470,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,69,1,0,0,0,0,0,0,1,0,...,161.5,70.4,70,105,0,0,1,0,522,1
9996,37,0,0,0,0,0,0,1,0,0,...,165.7,85.0,79,129,0,1,1,0,107,0
9997,31,1,0,0,0,0,0,1,0,0,...,151.5,61.3,69,123,99,0,0,1,66,0
9998,85,0,0,0,0,0,0,0,0,0,...,159.5,54.3,84,68,1,1,1,0,156,0


In [139]:
test

,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,T1c,...,Smoke,Height,Weight,FEV1_FVC_P,DLCO_VA_P,EGFR mutation Detection,Operation,Chemotherapy,Radiation Therapy,Survival period
0,35,1,0,0,0,0,0,0,0,0,...,1,174.2,69.1,82,43,0,0,1,0,355
1,60,1,0,0,0,0,0,0,1,0,...,0,158.8,67.4,52,86,0,0,0,1,166
2,45,1,1,1,1,0,0,0,0,0,...,0,159.7,71.2,49,68,99,1,1,1,623
3,31,0,0,0,1,0,0,0,0,0,...,1,153.6,71.0,82,87,1,1,0,0,445
4,55,0,0,1,0,0,0,1,0,0,...,1,163.6,72.0,68,127,99,0,1,0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,47,1,1,0,0,0,0,0,0,1,...,0,147.6,79.0,40,79,99,1,0,0,104
4996,44,1,0,1,0,0,1,0,0,0,...,2,151.0,41.6,69,67,99,0,0,0,42
4997,80,0,0,1,0,0,0,1,0,0,...,0,157.8,58.2,40,137,0,1,1,0,190
4998,69,1,0,0,0,0,0,0,0,0,...,2,167.5,56.7,89,77,0,1,1,1,635


In [140]:
train.isnull().sum()[train.isnull().sum()>0], test.isnull().sum()[test.isnull().sum()>0]

(Series([], dtype: int64), Series([], dtype: int64))

##### Experiment with XGBoost

In [141]:
X_raw = train.iloc[:,:-1]
y_raw = train.iloc[:,-1]
X_train_not_scaled, X_test_not_scaled, y_train_not_scaled, y_test_not_scaled = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42,stratify=y_raw)

###### Not scaled

In [142]:
dtrain_not_scaled = xgb.DMatrix(X_train_not_scaled, label=y_train_not_scaled)
dtest_not_scaled = xgb.DMatrix(X_test_not_scaled, label=y_test_not_scaled)

# params = {
#     'max_depth': 6, 
#     'eta': 0.3,  
#     'objective': 'binary:logistic',
#     'eval_metric': 'error' 
# }
params = {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.9}


num_rounds = 1500
model_not_scaled = xgb.train(params, dtrain_not_scaled, num_rounds)

preds = model_not_scaled.predict(dtest_not_scaled)
pred_labels_not_scaled = [round(pred) for pred in preds]

accuracy_not_scaled = accuracy_score(y_test_not_scaled, pred_labels_not_scaled)
print("num_rounds:", num_rounds)
print("Accuracy:", accuracy_not_scaled)
list(pred_labels_not_scaled).count(1)




num_rounds: 1500
Accuracy: 0.77


57

In [143]:
preds = model_not_scaled.predict(dtest_not_scaled)
pred_labels = [round(pred) for pred in preds]
len(pred_labels)

2000

In [144]:
preds = model_not_scaled.predict(X_test_not_scaled,iteration_range=(0,100))
pred_labels = [round(pred) for pred in preds]

TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'pandas.core.frame.DataFrame'>)

In [ ]:
set(model_not_scaled.predict(X_test_not_scaled))

{0}

###### Scaled

In [ ]:
bool_col_train = []
nume_col_train = []
bool_col_test = []
nume_col_test = []

for col in train.columns[:-1]:
    unique_values = train[col].unique()
    if len(unique_values) == 2 and all(value in [0, 1] for value in unique_values):
        bool_col_train.append(col)
    else:
        nume_col_train.append(col)

for col in test.columns:
    unique_values = test[col].unique()
    if len(unique_values) == 2 and all(value in [0, 1] for value in unique_values):
        bool_col_test.append(col)
    else:
        nume_col_test.append(col)


In [ ]:
nume_col_train, bool_col_train

(['AGE',
  'Type of Drink',
  'Smoke',
  'Height',
  'Weight',
  'FEV1_FVC_P',
  'DLCO_VA_P',
  'EGFR mutation Detection',
  'Survival period'],
 ['Adenocarcinoma',
  'Large cell carcinoma',
  'Squamous cell carcinoma',
  'TX',
  'T0',
  'T1',
  'T1a',
  'T1b',
  'T1c',
  'T2',
  'T2a',
  'T2b',
  'T3',
  'T4',
  'N1',
  'N2',
  'N3',
  'M1a',
  'M1b',
  'M1c',
  'Operation',
  'Chemotherapy',
  'Radiation Therapy'])

In [ ]:
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
scaler = MinMaxScaler()
for col in nume_col_train:
    npy = train[col].to_numpy().reshape(-1,1)
    normalized_data = scaler.fit_transform(npy)
    train_copy[col] = normalized_data

for col in nume_col_test:
    npy = test[col].to_numpy().reshape(-1,1)
    normalized_data = scaler.fit_transform(npy)
    test_copy[col] = normalized_data


In [ ]:
train_copy

,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,T1c,...,Height,Weight,FEV1_FVC_P,DLCO_VA_P,EGFR mutation Detection,Operation,Chemotherapy,Radiation Therapy,Survival period,Death
0,0.719298,1,1,0,1,0,0,0,0,0,...,0.000000,0.478814,0.823529,0.559701,1.000000,1,0,0,0.197479,0
1,0.614035,1,0,0,0,0,1,0,0,0,...,0.807775,0.378531,0.338235,0.634328,1.000000,0,1,0,0.771709,0
2,0.771930,1,1,0,1,0,0,0,0,0,...,0.600432,0.557910,0.705882,0.343284,1.000000,0,1,0,0.331933,0
3,0.807018,1,0,1,0,0,1,0,0,0,...,0.356371,0.632768,0.691176,0.820896,0.010101,0,0,0,0.350140,0
4,0.947368,1,0,0,0,0,0,0,0,1,...,0.654428,0.141243,0.250000,0.432836,1.000000,1,0,1,0.635854,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.666667,1,0,0,0,0,0,0,1,0,...,0.421166,0.509887,0.573529,0.582090,0.000000,0,1,0,0.708683,1
9996,0.105263,0,0,0,0,0,0,1,0,0,...,0.511879,0.716102,0.705882,0.761194,0.000000,1,1,0,0.127451,0
9997,0.000000,1,0,0,0,0,0,1,0,0,...,0.205184,0.381356,0.558824,0.716418,1.000000,0,0,1,0.070028,0
9998,0.947368,0,0,0,0,0,0,0,0,0,...,0.377970,0.282486,0.779412,0.305970,0.010101,1,1,0,0.196078,0


In [ ]:
X_preprocessed = train_copy.iloc[:,:-1]
y_preprocessed = train_copy.iloc[:,-1]

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_preprocessed, y_preprocessed, test_size=0.2, random_state=42,stratify=y_preprocessed)

In [152]:
dtrain_scaled = xgb.DMatrix(X_train_scaled, label=y_train_scaled)
dtest_scaled = xgb.DMatrix(X_test_scaled, label=y_test_scaled)

# params = {
#     'max_depth': 6,  # 트리의 최대 깊이
#     'eta': 0.3,  # 학습률
#     'objective': 'binary:logistic',
#     'eval_metric': 'error'  # 평가 지표 설정
# }

params = {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.9}
num_rounds = 1600
model_scaled = xgb.train(params, dtrain_scaled, num_rounds)

preds = model_scaled.predict(dtest_scaled)
pred_labels_scaled = [round(pred) for pred in preds]

accuracy_scaled = accuracy_score(y_test_scaled, pred_labels_scaled)
print("Accuracy:", accuracy_scaled)
list(pred_labels_scaled).count(1)


Accuracy: 0.7695


60

In [ ]:
preds = XGB_model_scaled.predict(X_test_scaled,iteration_range=(0,100))
# pred_labels = [round(pred) for pred in preds]

# set(pred_labels)
preds
# print(f"Accuracy:{XGB_model_scaled.score(X_test_scaled,y_test_scaled)}")
# print(f"Prediction:{XGB_model_scaled.predict(X_test_scaled,iteration_range=(0,100))}")

array([0.25657046, 0.19014987, 0.12870894, ..., 0.19107695, 0.2166219 ,
       0.23703489], dtype=float32)

In [ ]:
lst = XGB_model_scaled.predict_proba(X_test_scaled)
lst


array([[0.7626487 , 0.23735127],
       [0.81589365, 0.18410632],
       [0.8317123 , 0.1682877 ],
       ...,
       [0.82381916, 0.17618087],
       [0.829104  , 0.17089598],
       [0.7158649 , 0.28413513]], dtype=float32)

In [ ]:
[0 if i>=0.5 else 1 for i in XGB_model_scaled.predict_proba(X_test_scaled)]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
lst.shape

(2000, 2)

#### Submission

In [145]:
real_test_not_scaled = xgb.DMatrix(test)

In [149]:
submission = pd.read_csv('submission.csv')
# submission['Death'] = ...
submission['Death'] = pred_labels
submission['Death'].unique()

array([0, 1], dtype=int64)

In [150]:
submission.to_csv('submission.csv',index=False)

In [147]:
preds = model_not_scaled.predict(real_test_not_scaled)
pred_labels = [round(pred) for pred in preds]
list(pred_labels).count(1)

179

In [151]:
!explorer .